# Homework 2

# Setup

### Our usual imports and initializing napari

In [1]:
import numpy as np
import pandas as pd
import napari
import tifffile
import skimage as ski
import scipy.ndimage as ndi
import glob
import plotly.express as px
import cellpose.models as models
import matplotlib.pyplot as plt
import cv2
import dask
import sutils

In [2]:
viewer = napari.Viewer()

## Part 1:  Quantify nucleii in "Easy.tif"

Use similar analysis (rolling ball background subtraction, gaussian blur, threshold, label, regionprops_table) to make a label image of the nucleii in "Easy.tif".  

In [3]:
easy_img = ski.io.imread('files/Easy.tif')
backsub_img = sutils.backsub_2D(easy_img, radius=20)
blurred_img = ndi.gaussian_filter(backsub_img, 2)
thresholded_img = blurred_img>10
label_img, number_objects = ndi.label(thresholded_img)

viewer.layers.clear()
viewer.add_image(easy_img, name='easy_img', colormap='gray', blending='additive')
viewer.add_image(backsub_img, name='backsub_img', colormap='gray', blending='additive')
viewer.add_image(blurred_img, name='blurred_img', colormap='gray', blending='additive')
viewer.add_image(thresholded_img, name='thresholded_img', colormap='gray', blending='additive')
viewer.add_labels(label_img, name='label_img')

<Labels layer 'label_img' at 0x1f43bc2abc0>

Note that a lot of objects are merged that are single objects, we will find ways using watershed to clean this up later, but for now let's filter objects on size. 

Use the ski.measure.regionprops_table() to generate a results table and the ski.util.map_array() function to make "area_img" and then visualize this in napari.

In [4]:
results = pd.DataFrame(ski.measure.regionprops_table(label_img, easy_img, properties=('label', 'area', 'centroid', 'mean_intensity')))
area_img = ski.util.map_array(label_img, results['label'].values, results['area'].values)


In [5]:
viewer.add_image(area_img, name='area_img', colormap='blue', blending='additive')

<Image layer 'area_img' at 0x1f48b532cb0>

## Part 2:  Filter the nucleii by size

### Finding a minimum size

We want to find area limits that let through objects that are single nucleii, but filter out objects that are multiple nucleii and garbage.  

To find the low end (ie the MINIMUM size threshold) we can use our area_img.  Let's just adjust the contrast until the objects that are too small disappear and we are left only with the good ones.

To see the VALUE of "contrast" being used (which for us is the object's area), we can right click on the "contrast limits" slider.

Set a variable called "min_area" to the value you found.

In [6]:
min_area = 35

### Finding a maximum size

Now we want to find a "max_area" that filters out the double nucleii.

To do this we can play a trick, we can use napari's "gray_r" colormap to visualize the inverse of the area_img.  First we have to play another trick, where we set the area_img's intensity values to be high wherever there is no object (otherwise the background will show as bright).  We will do this using area_img[area_img==0] = np.max(area_img)

area_img==0 returns a binary image, where only the pixels that had value equal to 0 are True and all others are false.  When we ask area_img[..] on a binary image, numpy is smart and returns only the pixels that are True.  We then set all of these pixels to the maximum value in the image.

In [7]:
### THIS CODE STAYS IN THE STUDENT VERSION
area_img[area_img==0] = np.max(area_img)
viewer.add_image(area_img, name='area_img', colormap='gray_r', blending='additive')

<Image layer 'area_img [1]' at 0x1f48b59a050>

We want to find a contrast that lets through all the single nucleii, but filters out the double nucleii.  By right clicking on "contrast limits" we can see the actual values of the contrast limits.  Adjusting the upper level is effectively adjusting the area of objects that we are allowing to be display.


Once you find this contrast, use it to filter the results table to only include objects that are smaller than the best max area.

In [8]:
max_area = 250

Alternatively, you can just guess a max_area, and see what results you get below.

### Filtering the objects

Finally, use the "sutils.remove_objects" function to keep only the objects that are small enough to be single nucleii, then show this new labeled image in napari.

In [9]:
filtered_labels = sutils.remove_objects(label_img, min_area, max_area)
viewer.add_labels(filtered_labels, name='filtered_labels')

<Labels layer 'filtered_labels' at 0x1f48b532da0>

## Part 3 (extra credit): Looking at object standard deviations

### Standard deviation of original labels

Recall we can create our own functions for quantifying objects with regionprops, this function finds the standard deviation of the intensity of each object.

In [10]:
### DO NOT REMOVE FROM STUDENT VERSION

def object_stdeviation(mask_img, intensity_img):
    return np.std(intensity_img[mask_img])


Use this function and the "extra_properties" argument of ski.measure.regionprops_table to find the standard deviation of the intensities of each nucleii (in addition to the usual 'label', 'area', 'centroid', 'mean_intensity').  Recall that if you want to quantify intensities, regionprops_table needs both the label image and the intensity image.

In [11]:
results_dict = ski.measure.regionprops_table(filtered_labels, easy_img, properties=('label', 'area', 'centroid', 'mean_intensity'), extra_properties=[object_stdeviation])
results = pd.DataFrame(results_dict)

In [12]:
results

,label,area,centroid-0,centroid-1,mean_intensity,object_stdeviation
0,1,100.0,3.350000,30.370000,38.990000,16.196601
1,7,133.0,11.255639,80.526316,38.526316,17.693964
2,9,199.0,19.241206,177.447236,44.532663,19.951390
3,10,147.0,18.836735,102.632653,39.591837,21.179747
4,11,193.0,23.404145,131.088083,46.435233,26.302292
...,...,...,...,...,...,...
148,215,187.0,507.550802,94.978610,40.732620,17.477354
149,216,198.0,507.929293,396.858586,53.449495,35.524710
150,217,106.0,508.754717,51.113208,44.311321,30.254070
151,218,72.0,508.555556,228.500000,44.486111,25.854826


In [13]:
### DO NOT REMOVE FROM STUDENT VERSION
results['SNR'] = results['mean_intensity']/results['object_stdeviation']
results['Type'] = 'Regular'

SNR is a quick way of evaluating how strong our signal to noise is.  Looking at our labels we can see that a lot of pixels that are associated with a nucleus are on the edge where there really is not nucleus.  Including these pixels is going to make our standard deviation much higher that it probably actually is.

### Shrinking the labels

Create a new variable:  shrunk_labels, takes the filtered_labels and shrinks them by 1 pixel.  Add it to napari to make sure it worked.

In [14]:
shrunk_labels = sutils.shrink_labels(filtered_labels, shrinkage=5)
viewer.add_labels(shrunk_labels, name='shrunk_labels')

<Labels layer 'shrunk_labels' at 0x1f43bc3c370>

Create a new shrunk_results table that is the regionprops_table of the shrunk_labels, including the object_stdeviation as well.

In [15]:
shrunk_results_dict = ski.measure.regionprops_table(shrunk_labels, easy_img, properties=('label', 'area', 'centroid', 'mean_intensity'), extra_properties=[object_stdeviation])
shrunk_results = pd.DataFrame(shrunk_results_dict)

In [16]:
results

,label,area,centroid-0,centroid-1,mean_intensity,object_stdeviation,SNR,Type
0,1,100.0,3.350000,30.370000,38.990000,16.196601,2.407295,Regular
1,7,133.0,11.255639,80.526316,38.526316,17.693964,2.177370,Regular
2,9,199.0,19.241206,177.447236,44.532663,19.951390,2.232058,Regular
3,10,147.0,18.836735,102.632653,39.591837,21.179747,1.869325,Regular
4,11,193.0,23.404145,131.088083,46.435233,26.302292,1.765444,Regular
...,...,...,...,...,...,...,...,...
148,215,187.0,507.550802,94.978610,40.732620,17.477354,2.330594,Regular
149,216,198.0,507.929293,396.858586,53.449495,35.524710,1.504572,Regular
150,217,106.0,508.754717,51.113208,44.311321,30.254070,1.464640,Regular
151,218,72.0,508.555556,228.500000,44.486111,25.854826,1.720612,Regular


In [17]:
shrunk_results

,label,area,centroid-0,centroid-1,mean_intensity,object_stdeviation
0,1,54.0,2.537037,30.333333,51.777778,5.613036
1,7,59.0,11.254237,80.525424,55.389831,4.528692
2,9,109.0,19.247706,177.477064,60.678899,6.949201
3,10,71.0,18.887324,102.676056,59.295775,9.096346
4,11,104.0,23.413462,131.086538,68.769231,11.492666
...,...,...,...,...,...,...
148,215,119.0,508.453782,94.890756,49.218487,14.563197
149,216,118.0,508.889831,396.720339,74.635593,31.023137
150,217,56.0,509.589286,50.928571,67.839286,22.875265
151,218,34.0,509.294118,228.500000,69.500000,10.541933


In [18]:
### DO NOT REMOVE FROM STUDENT VERSION
shrunk_results['SNR'] = shrunk_results['mean_intensity']/shrunk_results['object_stdeviation']
shrunk_results['Type'] = 'Shrunk'

### Plotting results of SNR

If all went well we should get a nice plot of the SNR in shrunk vs unshrunk labels.

In [19]:
### DO NOT REMOVE FROM STUDENT VERSION

stapled_results = pd.concat([results, shrunk_results])
px.box(stapled_results, x='Type', y='SNR', points='all', width=400)


By shrinking our objects so that they only included pixels from the nucleus and not the borders, we drastically increased the signal to noise ratio.